In [1]:
# %pip install tensorflow gpflow

In [2]:
print('hello world')

hello world


In [3]:
# %pip install tensorflow --upgrade

In [4]:
# pip install --upgrade yfinance 

In [23]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta

def fetch_and_process_data(
    ticker="BTC-USD",
    start_date=(datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
    end_date=datetime.now().strftime('%Y-%m-%d')
):
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    
    # Ensure 'Close' is a Series by creating a copy
    close = data["Close"].copy()  # This is now a Series
    close.name = ticker    # Add name to Series
    returns = close.pct_change().dropna()
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    
    # Now we ensure we're using the same index for both series
    valid_index = returns.index
    return close.loc[valid_index], pd.Series(std_returns.flatten(), index=valid_index, name="Standardized Returns")

def generate_signals(returns, train_ratio=0.8, epochs=50):
    # Prepare features
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # GPR for trend detection
    kernel = Matern32()
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns + trend as features
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    
    # Train/test split
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    test_features = features[split_index:]
    train_labels = returns.values[:split_index]
    test_labels = returns.values[split_index:]
    
    # Reshape for LSTM
    train_features = train_features.reshape(
        (train_features.shape[0], 1, train_features.shape[1])
    )
    test_features = test_features.reshape(
        (test_features.shape[0], 1, test_features.shape[1])
    )
    
    # Build LSTM model
    model = Sequential(
        [
            LSTM(
                64,
                return_sequences=True,
                input_shape=(train_features.shape[1], train_features.shape[2]),
            ),
            Dropout(0.2),
            LSTM(32),
            Dense(1, activation="tanh"),
        ]
    )
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    
    # Fit the model (using next-day return as a rough training signal)
    model.fit(train_features, train_labels, epochs=epochs, batch_size=32, verbose=0)
    
    # Predict signals for all data (train + test)
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Convert continuous signals to [-1, 1] range
    positions = np.clip(predicted_signals, -1, 1)
    return positions

def backtest_strategy(close, positions):
    # Convert positions to entry/exit signals by ensuring that close is a Series with a name
    positions = pd.Series(positions, index=close.index, name=close.name)
    entries = positions > 0
    exits = positions < 0
    
    # Create portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=close, entries=entries, exits=exits, size=np.abs(positions), freq="1D"
    )
    return pf

def plot_portfolio_performance(portfolio):
    # Get the symbol name from the portfolio, assuming portfolio.close is a Series with a name
    symbol = portfolio.close.name
    
    # Create a plot for portfolio value
    fig = portfolio.plot_value(
        column=symbol,
        title=f'Portfolio Value Over Time - {symbol}'
    )
    fig.show()
    
    # Create a plot for drawdown
    fig = portfolio.plot_drawdown(
        column=symbol,
        title=f'Portfolio Drawdown - {symbol}'
    )
    fig.show()

def main():
    # Set the symbol
    symbol = "BTC-USD"
    
    try:
        # Get data for the specified period
        close, returns = fetch_and_process_data(
            ticker=symbol,
            start_date="2023-01-01",  # Historical start date
            end_date="2024-01-01"
        )
        
        # Generate trading signals (adjust epochs as needed)
        positions = generate_signals(returns, train_ratio=0.8, epochs=50)
        
        # Run backtest with generated signals
        portfolio = backtest_strategy(close, positions)
        
        # Print key portfolio metrics
        print("\nPortfolio Metrics:")
        print(f"Total Return: {float(portfolio.total_return(column=symbol)):.2%}")
        print(f"Sharpe Ratio: {float(portfolio.sharpe_ratio(column=symbol)):.2f}")
        print(f"Max Drawdown: {float(portfolio.max_drawdown(column=symbol)):.2%}")
        
        # Plot portfolio performance and drawdown
        plot_portfolio_performance(portfolio)
        
        # Print detailed statistics
        stats = portfolio.stats(column=symbol)
        print("\nDetailed Statistics:")
        for key, value in stats.items():
            if isinstance(value, (float, np.float64)):
                print(f"{key}: {value:.4f}")
            else:
                print(f"{key}: {value}")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



An error occurred: 'DataFrame' object has no attribute 'name'


AttributeError: 'DataFrame' object has no attribute 'name'

In [24]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta

def fetch_and_process_data(
    ticker="BTC-USD",
    start_date=(datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
    end_date=datetime.now().strftime('%Y-%m-%d')
):
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    
    # Use "Adj Close" if available, otherwise fallback to "Close"
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    
    # Ensure price_series is a Series
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def generate_signals(returns, train_ratio=0.8, epochs=50):
    # Prepare features: using time step as a feature
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use GPR for trend detection
    kernel = Matern32()
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form features
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    
    # Train/test split (only use train_features for training)
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape for LSTM input (3D shape required)
    train_features = train_features.reshape(
        (train_features.shape[0], 1, train_features.shape[1])
    )
    
    # Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    
    # Fit the model on training portion
    model.fit(train_features, train_labels, epochs=epochs, batch_size=32, verbose=0)
    
    # Predict signals using all features
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Clip predictions to the range [-1, 1]
    positions = np.clip(predicted_signals, -1, 1)
    
    return positions

def backtest_strategy(price_series, positions):
    # Ensure price_series is a Series with a name
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    
    # Create the portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=price_series, 
        entries=entries, 
        exits=exits, 
        size=np.abs(positions), 
        freq="1D"
    )
    return pf

def evaluate_performance(portfolio):
    # Since portfolio is for a single asset, call the performance methods directly
    print("\nPortfolio Metrics:")
    print(f"Total Return: {float(portfolio.total_return()):.2%}")
    print(f"Sharpe Ratio: {float(portfolio.sharpe_ratio()):.2f}")
    print(f"Max Drawdown: {float(portfolio.max_drawdown()):.2%}")
    
    # Plot portfolio value
    fig_value = portfolio.plot_value(
        title=f'Portfolio Value Over Time - {portfolio.close.name}'
    )
    fig_value.show()
    
    # Use plot_drawdowns (note the plural) for drawdown plotting
    fig_drawdowns = portfolio.plot_drawdowns(
        title=f'Portfolio Drawdowns - {portfolio.close.name}'
    )
    fig_drawdowns.show()

def main():
    ticker = "BTC-USD"
    
    # Fetch and process data
    price_series, returns = fetch_and_process_data(
        ticker=ticker, 
        start_date="2024-01-01", 
        end_date="2025-01-01"
    )
    
    # Generate trading signals
    positions = generate_signals(returns, train_ratio=0.8, epochs=50)
    
    # Backtest strategy
    portfolio = backtest_strategy(price_series, positions)
    
    # Evaluate performance
    evaluate_performance(portfolio)
    
    print(portfolio.stats())
 
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




Portfolio Metrics:
Total Return: 50.00%
Sharpe Ratio: 1.34
Max Drawdown: -25.38%


Start                               2024-01-02 00:00:00
End                                 2024-12-31 00:00:00
Period                                365 days 00:00:00
Start Value                                       100.0
End Value                                    150.004092
Total Return [%]                              50.004092
Benchmark Return [%]                         107.814556
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              25.379407
Max Drawdown Duration                 213 days 00:00:00
Total Trades                                         95
Total Closed Trades                                  94
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  37.234043
Best Trade [%]                                26.811372
Worst Trade [%]                               -6

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import gpflow
from gpflow.utilities import set_trainable
from gpflow.models import GPR
from gpflow.kernels import Matern32
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import vectorbt as vbt


# ------------------------------------------------------
# 1. Data Preprocessing
# ------------------------------------------------------
def fetch_data(ticker="AAPL", start="2020-01-01", end="2021-12-31"):
    """
    Fetch historical price data from Yahoo Finance, then compute daily returns
    and standardize them. Extract momentum indicators to form a feature set.
    """
    data = yf.download(ticker, start=start, end=end)
    data.dropna(inplace=True)

    # Compute daily returns
    data["Returns"] = data["Adj Close"].pct_change().fillna(0.0)

    # Simple momentum indicator (e.g., 10-day rolling return)
    data["Momentum_10"] = data["Adj Close"].pct_change(periods=10).fillna(0.0)
    data["Momentum_20"] = data["Adj Close"].pct_change(periods=20).fillna(0.0)

    # Standardize numeric columns except 'Adj Close'
    scaler = StandardScaler()
    feature_cols = ["Returns", "Momentum_10", "Momentum_20"]
    data[feature_cols] = scaler.fit_transform(data[feature_cols])

    # Drop any rows with NaNs that might remain
    data.dropna(inplace=True)

    return data


# ------------------------------------------------------
# 2. Changepoint Detection (CPD) using Gaussian Processes
# ------------------------------------------------------
def detect_changepoints(features):
    """
    Fits a Gaussian Process (GP) model to detect changepoints in the feature data.
    For simplicity, this function focuses on a single feature (e.g., Returns).
    In practice, you could expand this to multi-dimensional CPD.
    """
    X = np.arange(len(features)).reshape(-1, 1).astype(float)
    y = features.values.reshape(-1, 1).astype(float)

    # Define a Matern 3/2 kernel
    kernel = Matern32()
    gpr_model = GPR(data=(X, y), kernel=kernel, mean_function=None)

    # Typically, we may fix the likelihood variance for stable GP estimation
    set_trainable(gpr_model.likelihood.variance, False)

    # Optimize the GP hyperparameters
    opt = gpflow.optimizers.Scipy()
    opt.minimize(
        gpr_model.training_loss,
        variables=gpr_model.trainable_variables,
        options=dict(maxiter=50),
    )

    # Predict the function mean which we can interpret as a "trend" or baseline
    mean, var = gpr_model.predict_f(X)

    # Convert the EagerTensor to a NumPy array, then flatten
    mean_np = mean.numpy()
    # A simple changepoint "severity" measure: gradient magnitude or variance
    # Here, we use the absolute difference of the predicted mean as a placeholder
    cp_severity = np.abs(np.diff(mean_np.flatten(), prepend=mean_np[0]))

    return mean_np.flatten(), cp_severity


# ------------------------------------------------------
# 3. Deep Momentum Network (DMN) with an LSTM
#    - Optimizing for Sharpe ratio
# ------------------------------------------------------
def custom_sharpe_loss(y_true, y_pred):
    """
    Custom loss function approximating negative Sharpe ratio.
    y_pred: predicted position sizes
    y_true: actual returns (or some target based on returns).
    """
    # Safe epsilon to avoid division by zero
    eps = 1e-9

    # Calculate portfolio returns = predicted_position * actual_return
    portfolio_returns = y_pred * y_true
    mean_return = tf.reduce_mean(portfolio_returns)
    std_return = tf.math.reduce_std(portfolio_returns) + eps

    # We minimize negative Sharpe ratio => -(mean / std)
    return -(mean_return / std_return)


def build_dmn(input_shape):
    """
    Builds an LSTM-based model (DMN) that outputs position sizes (between -1 and 1).
    """
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dense(1, activation="tanh"))  # Position range [-1, 1]
    # Use a custom loss approximating negative Sharpe ratio
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_sharpe_loss
    )
    return model


# ------------------------------------------------------
# 4. VectorBT Backtesting Integration
# ------------------------------------------------------
def backtest_with_vectorbt(prices, positions):
    """
    Backtest the DMN signals using vectorbt's Portfolio.from_signals.
    We interpret positions > 0 as buys, positions < 0 as sells.
    The absolute value of the position determines the size.
    """
    # Convert positions array to a Pandas Series with the same index as prices
    positions_series = pd.Series(positions, index=prices.index)

    # Generate entry (positions > 0) and exit (positions < 0) signals
    entries = positions_series > 0
    exits = positions_series < 0

    # Build the portfolio
    # size=np.abs(positions) for partial positions if needed
    portfolio = vbt.Portfolio.from_signals(
        close=prices,
        entries=entries,
        exits=exits,
        size=np.abs(positions_series),
        freq="1D",
    )

    return portfolio


# ------------------------------------------------------
# Main Execution
# ------------------------------------------------------
def main():
    # 1. Data Preprocessing
    data = fetch_data(ticker="BTC-USD", start="2020-01-01", end="2024-12-31")

    # Target feature for CPD (e.g., standardized returns)
    returns_feature = data["Returns"]

    # 2. Changepoint Detection
    trend, cp_severity = detect_changepoints(returns_feature)

    # Add CPD outputs as new features
    data["Trend"] = trend
    data["CP_Severity"] = cp_severity

    # 3. Build and Train DMN
    # Prepare feature set for DMN: returns, momentum, CPD, etc.
    feature_cols = ["Returns", "Momentum_10", "Momentum_20", "Trend", "CP_Severity"]
    X = data[feature_cols].values
    y = data["Returns"].values  # Using returns as a training signal

    # Reshape for LSTM: (batch_size, time_steps=1, input_dim=len(feature_cols))
    X_reshaped = X.reshape((X.shape[0], 1, X.shape[1]))

    model = build_dmn(input_shape=(1, X.shape[1]))
    model.fit(X_reshaped, y, epochs=30, batch_size=32, verbose=1)

    # Generate signals (predicted position sizes)
    positions = model.predict(X_reshaped).flatten()

    # Use vectorbt for backtesting
    portfolio = backtest_with_vectorbt(data["Adj Close"], positions)

    # Print key metrics from vectorbt
    print("\nVectorBT Performance Metrics:")
    print(f"Total Return: {portfolio.total_return():.2%}")
    print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
    print(f"Max Drawdown: {portfolio.max_drawdown():.2%}")

    # Plot the equity curve
    portfolio.plot().show()

    print(portfolio.stats())

if __name__ == "__main__":
    main()

In [25]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta

def fetch_and_process_data(
    ticker="BTC-USD",
    start_date="2020-01-01",
    end_date="2020-12-31"
):
    # Download data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    if data.empty:
        raise ValueError(f"No data found for {ticker} between {start_date} and {end_date}")
    
    # Use "Adj Close" if available, otherwise fallback to "Close"
    if "Adj Close" in data.columns:
        price_series = data["Adj Close"].copy()
    else:
        price_series = data["Close"].copy()
    
    # Ensure price_series is a Series
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    price_series.name = ticker
    returns = price_series.pct_change().dropna()
    
    scaler = StandardScaler()
    std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
    
    valid_index = returns.index
    return price_series.loc[valid_index], pd.Series(
        std_returns.flatten(), index=valid_index, name="Standardized Returns"
    )

def generate_signals(returns, train_ratio=0.8, epochs=50):
    # Prepare features: using time step as a feature
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use GPR for trend detection
    kernel = Matern32()
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form features
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    
    # Train/test split (only use train_features for training)
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape for LSTM input (3D shape required)
    train_features = train_features.reshape(
        (train_features.shape[0], 1, train_features.shape[1])
    )
    
    # Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(0.2),
        LSTM(32),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    
    # Fit the model on training portion
    model.fit(train_features, train_labels, epochs=epochs, batch_size=32, verbose=0)
    
    # Predict signals using all features
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Clip predictions to the range [-1, 1]
    positions = np.clip(predicted_signals, -1, 1)
    
    return positions

def backtest_strategy(price_series, positions):
    # Ensure price_series is a Series with a name
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    positions = pd.Series(positions, index=price_series.index, name=price_series.name)
    entries = positions > 0
    exits = positions < 0
    
    # Create the portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=price_series, 
        entries=entries, 
        exits=exits, 
        size=np.abs(positions), 
        freq="1D"
    )
    return pf

def evaluate_performance(portfolio, label=""):
    print(f"\nPortfolio Metrics for period: {label}")
    try:
        total_return = portfolio.total_return()
        sharpe = portfolio.sharpe_ratio()
        max_drawdown = portfolio.max_drawdown()
        print(f"Total Return: {float(total_return):.2%}")
        print(f"Sharpe Ratio: {float(sharpe):.2f}")
        print(f"Max Drawdown: {float(max_drawdown):.2%}")
    except Exception as e:
        print("Error calculating performance metrics:", e)
    
    # Optionally, plot the portfolio performance (comment these lines if running multiple tests to avoid multiple pop-ups)
    fig_value = portfolio.plot_value(title=f'Portfolio Value Over Time - {label}')
    fig_value.show()
    
    fig_drawdowns = portfolio.plot_drawdowns(title=f'Portfolio Drawdowns - {label}')
    fig_drawdowns.show()

def test_period(ticker, start_date, end_date, label, train_ratio=0.8, epochs=50):
    print(f"\nTesting period {label} ({start_date} to {end_date})")
    try:
        price_series, returns = fetch_and_process_data(ticker=ticker, start_date=start_date, end_date=end_date)
        positions = generate_signals(returns, train_ratio=train_ratio, epochs=epochs)
        portfolio = backtest_strategy(price_series, positions)
        evaluate_performance(portfolio, label=label)
        print(portfolio.stats())
    except Exception as e:
        print(f"Failed testing period {label}: {e}")

def main():
    ticker = "BTC-USD"
    current_date = datetime.now()
    current_year = current_date.year
    
    # Test full calendar years starting from 2019.
    for year in range(2019, current_year + 1):
        # For past years, use full year. For current year, use YTD.
        if year < current_year:
            start_date = f"{year}-01-01"
            end_date = f"{year}-12-31"
            label = f"{year}"
        else:
            start_date = f"{year}-01-01"
            end_date = current_date.strftime('%Y-%m-%d')
            label = f"{year} (YTD)"
        test_period(ticker, start_date, end_date, label)
    
    # Define multi-year tests ending on current day.
    multi_year_periods = {
        "2-Year": 2,
        "3-Year": 3,
        "5-Year": 5
    }
    
    for period_label, years in multi_year_periods.items():
        start_date_dt = current_date - timedelta(days=int(365 * years))
        start_date = start_date_dt.strftime('%Y-%m-%d')
        end_date = current_date.strftime('%Y-%m-%d')
        label = f"{period_label} Period ({start_date} to {end_date})"
        test_period(ticker, start_date, end_date, label)

if __name__ == "__main__":
    main()


Testing period 2019 (2019-01-01 to 2019-12-31)


[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




Portfolio Metrics for period: 2019
Total Return: 94.53%
Sharpe Ratio: 1.63
Max Drawdown: -21.78%


Start                               2019-01-02 00:00:00
End                                 2019-12-30 00:00:00
Period                                363 days 00:00:00
Start Value                                       100.0
End Value                                    194.534334
Total Return [%]                              94.534334
Benchmark Return [%]                          84.941362
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              21.784298
Max Drawdown Duration                 109 days 00:00:00
Total Trades                                         86
Total Closed Trades                                  86
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                   33.72093
Best Trade [%]                                20.588567
Worst Trade [%]                              -13

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...


Portfolio Metrics for period: 3-Year Period (2022-02-21 to 2025-02-20)
Total Return: 47.13%
Sharpe Ratio: 0.53
Max Drawdown: -42.83%


Start                               2022-02-22 00:00:00
End                                 2025-02-19 00:00:00
Period                               1094 days 00:00:00
Start Value                                       100.0
End Value                                     147.13235
Total Return [%]                               47.13235
Benchmark Return [%]                         152.404379
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              42.829705
Max Drawdown Duration                 592 days 00:00:00
Total Trades                                        289
Total Closed Trades                                 288
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  36.805556
Best Trade [%]                                26.811372
Worst Trade [%]                               -9

[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.




Portfolio Metrics for period: 5-Year Period (2020-02-22 to 2025-02-20)
Total Return: 6.10%
Sharpe Ratio: 0.25
Max Drawdown: -68.27%


Start                               2020-02-23 00:00:00
End                                 2025-02-19 00:00:00
Period                               1824 days 00:00:00
Start Value                                       100.0
End Value                                    106.098127
Total Return [%]                               6.098127
Benchmark Return [%]                         873.706053
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              68.266078
Max Drawdown Duration                1497 days 00:00:00
Total Trades                                        494
Total Closed Trades                                 493
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  36.916836
Best Trade [%]                                26.811372
Worst Trade [%]                              -37